In [1]:
# 

In [2]:
import h5pyd
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.spatial import cKDTree

# Open the desired year of nsrdb data
# server endpoint, username, password is found via a config file
f = h5py.File('dataset/tmy2020.h5', 'r')
list(f.keys())


['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'coordinates',
 'dew_point',
 'dhi',
 'dni',
 'fill_flag',
 'ghi',
 'meta',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'time_index',
 'tmy_year',
 'tmy_year_short',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

In [3]:
clearsky_ghidset = f['clearsky_ghi']
clearsky_ghidset_attr = clearsky_ghidset.attrs
clearsky_ghidset[0:10]
clearsky_ghidset.shape

(8760, 2018392)

In [4]:
for k,v in clearsky_ghidset_attr.items():
    print(str(k) + ' , ' + str(v))


data_source , output
elevation_correction , False
physical_max , 1350.0
physical_min , 0.0
psm_scale_factor , 1.0
psm_units , W/m2
scale_factor , 1.0
spatial_interp_method , nan
temporal_interp_method , nan
units , W/m2


In [5]:
wind_speed = f['wind_speed']
wind_speed_attr = wind_speed.attrs
wind_speed[0:10]
wind_speed.shape

air_temperature = f['air_temperature']
air_temperature_attr = air_temperature.attrs
air_temperature[0:10]
air_temperature.shape

(8760, 2018392)

In [106]:
# Get coordinates in USA only (2018392 items -> 546219 items)
meta = pd.DataFrame(f['meta'][...])
USA = meta.loc[meta['country'] == b'United States'] # Note .h5 saves strings as bit-strings
USA.head()
df_coord_usa = USA[['latitude', 'longitude']].copy()
df_coord_usa.shape
df_coord_usa.head(600000)

# dset = f['ghi']
# data = dset[0][USA.index]  # full-resolution subset

,latitude,longitude
14294,18.930000,-155.660004
14327,18.969999,-155.740005
14328,18.969999,-155.699997
14329,18.969999,-155.660004
14330,18.969999,-155.619995
...,...,...
2018343,51.889999,179.699997
2018344,51.970001,179.699997
2018345,51.930000,179.699997
2018350,51.930000,179.740005


In [137]:
dset_coords = f['coordinates'][...]

dset_names = \
['air_temperature',
 'alpha',
 'aod',
 'asymmetry',
 'cld_opd_dcomp',
 'cld_reff_dcomp',
 'clearsky_dhi',
 'clearsky_dni',
 'clearsky_ghi',
 'cloud_press_acha',
 'cloud_type',
 'dew_point',
 'dhi',
 'dni',
 'ghi',
 'ozone',
 'relative_humidity',
 'solar_zenith_angle',
 'ssa',
 'surface_albedo',
 'surface_pressure',
 'total_precipitable_water',
 'wind_direction',
 'wind_speed']

tree = cKDTree(dset_coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

ploc = (44.3142,-89.8964)
ploc_idx = nearest_site(tree, ploc[0], ploc[1] )
print(ploc_idx)
print(dset_coords[ploc_idx])

# df = pd.DataFrame(f['meta'][...])
# df['ghi'] = data / dset.attrs['psm_scale_factor']

dict_result = {}

dict_result['latitude'] = dset_coords[ploc_idx][0]
dict_result['longitude'] = dset_coords[ploc_idx][1]

for field in dset_names:
    print(field)
    dset = f[field]
    dict_result[field] = np.mean(dset[::1,ploc_idx] / dset.attrs['psm_scale_factor'])

#print(dict_result)

df = pd.DataFrame([dict_result])
pd.set_option('display.max_columns', None)
df.head()

851380
[ 44.33 -89.9 ]
air_temperature
alpha
aod
asymmetry
cld_opd_dcomp
cld_reff_dcomp
clearsky_dhi
clearsky_dni
clearsky_ghi
cloud_press_acha
cloud_type
dew_point
dhi
dni
ghi
ozone
relative_humidity
solar_zenith_angle
ssa
surface_albedo
surface_pressure
total_precipitable_water
wind_direction
wind_speed


,latitude,longitude,air_temperature,alpha,aod,asymmetry,cld_opd_dcomp,cld_reff_dcomp,clearsky_dhi,clearsky_dni,clearsky_ghi,cloud_press_acha,cloud_type,dew_point,dhi,dni,ghi,ozone,relative_humidity,solar_zenith_angle,ssa,surface_albedo,surface_pressure,total_precipitable_water,wind_direction,wind_speed
0,44.330002,-89.900002,6.296918,1.223825,0.143093,0.601451,4.038803,4.51524,48.160959,326.783105,222.768721,145.182877,3.335959,2.157877,64.6871,189.119521,165.141438,0.327714,77.923252,89.670564,0.937952,0.349945,975.253425,1.585708,205.509589,3.247991
